In [12]:
pip install seaborn

   ---------------------------------------- 0.0/294.6 kB ? eta -:--:--
   - -------------------------------------- 10.2/294.6 kB ? eta -:--:--
   -------- ------------------------------ 61.4/294.6 kB 812.7 kB/s eta 0:00:01
   ---------------------------------------- 294.6/294.6 kB 3.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [4]:
# ===================== IMPORTS/LIBRARIES =====================

import tensorflow as tf
from mtcnn import MTCNN
from pathlib import Path
import pandas as pd
import glob
import cv2
import numpy as np
import csv
import os
import ast
import pydot
import pydotplus
import graphviz
from datetime import datetime
import matplotlib.pyplot as plt

import tensorflow_addons as tfa

import time

from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import Callback

C:\Users\CG\miniconda3\envs\torch\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
C:\Users\CG\miniconda3\envs\torch\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.12.0 and strictly below 2.15.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.10.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you wa

In [2]:
# ===================== TRAINING HISTORY FUNCTIONS =====================
def historyToCsv():
    # convert the history.history dict to a pandas DataFrame:     
    hist_df = pd.DataFrame(history.history) 
    
    current_datetime = datetime.now()

    # Convert the datetime object to a string
    filename_friendly_datetime_string = current_datetime.strftime("%Y-%m-%d_%H-%M-%S")
    
    # save to csv: 
    hist_csv_file = 'history' + filename_friendly_datetime_string + '.csv'
    with open(hist_csv_file, mode='w') as f:
        hist_df.to_csv(f)

def csvToHistory(csv_filename):
    # Read the CSV file into a pandas DataFrame
    hist_df = pd.read_csv(csv_filename, index_col=0)

    # Convert the DataFrame to a dictionary
    history_dict = hist_df.to_dict(orient='list')

    return history_dict

In [8]:
# ===================== DROWSINESS MODEL =====================

# import multi-task model keras file (rename accordingly depending on what multitask model was trained)
multi_task_model = tf.keras.models.load_model('MultiTaskModel.keras')

# retain weights and remove top layer
output_layer = multi_task_model.get_layer('embedding').output

drowsiness_model = Model(inputs=multi_task_model.input, outputs=output_layer)

# drowsiness_model.summary()
# tf.keras.utils.plot_model(drowsiness_model)
from keras.utils.vis_utils import plot_model
# plot_model(drowsiness_model, to_file='drowsinessModel_plot.png', show_shapes=True, show_layer_names=True)

existing_output = drowsiness_model.output

reshaped_output = tf.keras.layers.Reshape((16, 16, 2))(existing_output)

spatial_attention = tf.keras.layers.Conv2D(2, (1, 1), activation='sigmoid', padding='same')(reshaped_output)
spatial_attention = tf.keras.layers.Softmax()(spatial_attention)
output_tensor = tf.keras.layers.Multiply()([reshaped_output, spatial_attention])

# Add Global Average Pooling layer
output_tensor = tf.keras.layers.GlobalAveragePooling2D()(output_tensor)

# Add output layer with two classes and softmax activation
predictions = tf.keras.layers.Dense(1, activation='sigmoid', name='drowsiness_output')(output_tensor)

# Create the new model with the modified top layers
drowsiness_model = Model(inputs=drowsiness_model.input, outputs=predictions)

drowsiness_model.compile(
    optimizer='adam',
    loss={
        'drowsiness_output': 'binary_crossentropy'
    },
    metrics={
        'drowsiness_output': [
                             tf.keras.metrics.Accuracy(name='accuracy'),
                             tf.keras.metrics.Precision(name='precision'),
                             tf.keras.metrics.Recall(name='recall'),
                             tfa.metrics.F1Score(num_classes=1, threshold=0.5)]
    }
)

drowsiness_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 224, 224, 64  1792        ['input_5[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv2 (Conv2D)          (None, 224, 224, 64  36928       ['block1_conv1[0][0]']           
                                )                                                           

In [4]:
# ===================== ORIGINAL DATA GEN CLASS =====================

class CustomDataGen(tf.keras.utils.Sequence):
    
    def __init__(self, df, X_col, y_col,
                 batch_size,
                 input_size=(224, 224, 3),
                 shuffle=True,
                 random_seed=None):  # Add a new parameter for random seed
        
        self.df = df.copy()
        self.X_col = X_col
        self.y_col = y_col
        self.batch_size = batch_size
        self.input_size = input_size
        self.shuffle = shuffle
        self.random_seed = random_seed  # Store the random seed
        
        self.n = len(self.df)
#         self.n_coords = 2  # Assuming landmark coordinates are 2-dimensional
        self.n_drowsiness = 1  # Assuming a single illuminance value
    
    def on_epoch_end(self):
        if self.shuffle:
            self.df = self.df.sample(frac=1, random_state=self.random_seed).reset_index(drop=True)  # Use the random seed
    
    def __get_input(self, path, target_size):
    
        image = tf.keras.preprocessing.image.load_img(path)
        image_arr = tf.keras.preprocessing.image.img_to_array(image)

        image_arr = tf.image.resize(image_arr, (target_size[0], target_size[1])).numpy()

        return image_arr / 255.
    
    def __get_output(self, label, output_type):
        # Assuming output_type is 'coordinates', 'illuminance', or 'adjusted_image_path'
#         if output_type == 'coordinates':
#             # Assuming label is a string containing a dictionary-like structure
#             # Safely evaluate the string as a literal dictionary using ast.literal_eval
#             coordinates_dict = ast.literal_eval(label)
            
#             # Extract x and y coordinates for each landmark
#             landmarks = ['left_eye', 'right_eye', 'nose', 'mouth_left', 'mouth_right']
#             coordinates_list = [coordinates_dict[landmark] for landmark in landmarks]
            
#             # Flatten the list and convert to numpy array
#             coordinates_array = np.array([coord for landmark_coords in coordinates_list for coord in landmark_coords])
            
# #             print("Shape of landmarks_array:", coordinates_array.shape)
            
#             # If there are exactly 10 values, return the array, otherwise raise an error
#             if len(coordinates_array) == 10:
#                 return coordinates_array
#             else:
#                 raise ValueError("Expected 10 coordinates, but found {}".format(len(coordinates_array)))
#         elif output_type == 'illuminance':
#             # Convert the illuminance value to a float
#             return float(label)
#         elif output_type == 'adjusted_image_path':
#             # Assuming label is the path to the adjusted image
#             return self.__get_input(label, self.input_size)
        if output_type == 'drowsiness':
            # Convert the illuminance value to a float
            return int(label)
    
    def __get_data(self, batches):
        # Generates data containing batch_size samples

        path_batch = batches[self.X_col['path']]
        
#         coords_batch = batches[self.y_col['coordinates']]
        illuminance_batch = batches[self.y_col['drowsiness']]
#         adjusted_image_path_batch = batches[self.y_col['adjusted_image_path']]

        X_batch = np.asarray([self.__get_input(x, self.input_size) for x in path_batch])

#         y0_batch = np.asarray([self.__get_output(y, 'coordinates') for y in coords_batch])
        y0_batch = np.asarray([self.__get_output(y, 'drowsiness') for y in illuminance_batch])
#         y2_batch = np.asarray([self.__get_output(y, 'adjusted_image_path') for y in adjusted_image_path_batch])

        return X_batch, [y0_batch]
    
    def __getitem__(self, index):
        batches = self.df[index * self.batch_size:(index + 1) * self.batch_size]
        X, y = self.__get_data(batches)

        # Print a few examples of the data
#         print("Sample X:", tf.shape(X[0]))  # Print the first example in the batch
#         print("Sample y[0] (landmarks):", tf.shape(y[0][0]))  # Print the first example in the landmarks output
#         print("Sample y[1] (illum):", tf.shape(y[1][0]))  # Print the first example in the illum output
#         print("Sample y[2] (adjusted_image_path):", tf.shape(y[2][0]))  # Print the first example in the adjusted_image_path output

        return X, y
    
    def __len__(self):
        return self.n // self.batch_size

In [5]:
def getDrowsiness(filename):
    index = filename.rfind('.')
    return filename[index - 1] if index >= 1 else None

In [6]:
# ===================== DATA GEN SETUP (DROWSINESS TASK) =====================

test_df = pd.read_csv(r".\training.csv") # path to test_data csv
test_df["Filename"] = "./data/Training/" + test_df["Filename"]

test_df["Drowsiness"] = test_df["Filename"].apply(getDrowsiness).astype(int)


eval_df = pd.read_csv(r".\evaluation.csv") # path to test_data csv
eval_df["Filename"] = "./data/Evaluation/" + eval_df["Filename"]

eval_df["Drowsiness"] = eval_df["Filename"].apply(getDrowsiness).astype(int)


test_gen = CustomDataGen(df=test_df, X_col={'path': 'Filename'}, y_col={'drowsiness': 'Drowsiness'}, batch_size=32, input_size=(224, 224, 3), random_seed=438)
eval_gen = CustomDataGen(df=eval_df, X_col={'path': 'Filename'}, y_col={'drowsiness': 'Drowsiness'}, batch_size=32, input_size=(224, 224, 3), random_seed=438)

In [7]:
evaluation_result = drowsiness_model.fit(test_gen, epochs=10, validation_data=eval_gen)

Epoch 1/10
940/940 [==============================] - 1070s 1s/step - loss: 0.7745 - accuracy: 0.0010 - precision: 0.5818 - recall: 0.8998 - f1_score: 0.7067 - val_loss: 0.6941 - val_accuracy: 0.0000e+00 - val_precision: 0.5084 - val_recall: 1.0000 - val_f1_score: 0.6741
Epoch 2/10
940/940 [==============================] - ETA: 0s - loss: 0.6783 - accuracy: 0.0000e+00 - precision: 0.5843 - recall: 0.9762 - f1_score: 0.7310

KeyboardInterrupt: 

In [32]:
# test_images_path = r'./data/testing'
# test_images_list = os.listdir(test_images_path)

# y_true = np.array([getDrowsiness(frame) for frame in test_images_list])

# img0_path = r'./data/testing/0'
# img0_list = os.listdir(img0_path)
# img0_arr = np.array(img0_list)
# img0_arr

# img1_path = r'./data/testing/1'
# img1_list = os.listdir(img1_path)
# img1_arr = np.array(img1_list)
# img1_arr

# y_true = np.append(img0_arr, img1_arr)
# y_true

array(['003_glasses_mix_0_0.jpg', '003_glasses_mix_10020_0.jpg',
       '003_glasses_mix_10050_0.jpg', ...,
       '037_sunglasses_mix_7170_1.jpg', '037_sunglasses_mix_7200_1.jpg',
       '037_sunglasses_mix_7230_1.jpg'], dtype='<U34')

In [9]:
from keras.preprocessing.image import ImageDataGenerator

main_directory = r'./data/Testing'

data_generator = ImageDataGenerator(
    rescale=1./255
)

# Create a data generator
X_test = data_generator.flow_from_directory(
    main_directory,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',  # Set to 'binary' since you have two classes
    shuffle=False  # Set to True for training, False for prediction
)

# Obtain the mapping between filenames and labels
filename_label_mapping = X_test.class_indices

# print(filename_label_mapping)

# Iterate over the generator to load all labels
all_labels = []
num_batches = len(X_test)
for i in range(num_batches):
    _, labels = X_test[i]
    all_labels.extend(labels)

print(f'num_batches: { num_batches}')
# Convert labels to a NumPy array
labels_array = np.array(all_labels)

# Convert numerical labels to class labels without periods
class_names = list(filename_label_mapping.keys())
y_true = [class_names[int(label)] for label in labels_array]
y_true = np.array(y_true, dtype=int)

Found 29727 images belonging to 2 classes.
num_batches: 929


In [10]:
import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming 'model' is your trained machine learning model
# Assuming 'X_test' is your testing feature data
# Assuming 'y_true' is the true labels of the testing data
y_pred = drowsiness_model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int) # prediction

# Create confusion matrix
# cm = confusion_matrix(y_true, y_pred)

# print(classification_report(y_true, y_pred))

 40/929 [>.............................] - ETA: 5:07

KeyboardInterrupt: 

In [6]:
import matplotlib.pyplot as plt
import numpy
from sklearn import metrics

confusion_matrix = metrics.confusion_matrix(y_true=y_true, y_pred=y_pred)

NameError: name 'y_pred' is not defined

In [60]:
print(cm)

[[    0 14718]
 [    0 15009]]


In [56]:
type(y_true)
# print(y_pred)

list